<h1 style="display: flex; flex-direction: row; align-items: center; justify-content: space-between; margin-top:0; margin-bottom:10px;">
    <div style="vertical-align: middle;">EO4BEE</div>
    <div><a href="http://www.master-cde.eu/" target="_blank">
        <img src="https://i.ibb.co/DCxYvvz/logos1.png" width="390px" style="float: right; vertical-align: middle;">
    </a></div>
</h1>
**This webapp provide a tool to visualize various earth observation data to support the decision-making process of beekeeping in Europe | [GitHub Repo](https://github.com/YuanWANG2662/EO4BEE-HEROKU.git) | [Contact Author @Yuan WANG](mailto:yuanwang2662@gmail.com)**
1. Select the earth observation variable that you want to explore. <strong><a href='#data_source'>(See details about the data sources)</a></strong>
2. Select the time period that you're interested in.
3. Drag the time slider to visualize the image at a specific time which you want to investigate. 
4. Click on the tool button and then the 'i' icon to inspect value at a specific location, or the plot icon to plot the change of the chosen variable over time at a specific location.
5. If you want to visualize the change smoothly as a gif, please use the create timelapse tool to create and download customized timelapse gif. <strong><a href='#timelapse'>(Go to Create Timelapse APP)</a></strong>



In [1]:
# import all the required libraries
import ee
import geemap
import ipywidgets as widgets
from datetime import datetime
from IPython.display import Image
import copy

In [2]:
# initialize the needed widgets

# default style
style = {'description_width': 'initial'}
layout_small = widgets.Layout(width='180px')

# layer selector dropdown box
layer = widgets.Dropdown(
    description='Select Layer:',
    options=['Annual Average Temperature','Monthly Average Temperature','Annual Total Precipitation',
             'Monthly Total Precipitation','NDVI','Landcover','Soil Moisture (yearly mean)','Soil Moisture (monthly mean)',
             'PM2.5 (yearly mean)', 'PM2.5 (monthly mean)', 'Sulphur Dioxide (yearly mean)','Sulphur Dioxide (monthly mean)'],
    value='Annual Average Temperature',
    style=style
)

# function to get the available time span for each dataset accordingly
def getAvailableYears(layer):
    if layer == 'Annual Average Temperature' or layer == 'Monthly Average Temperature':
        return ['2000', '2001', '2002', '2003', '2004', '2005','2006', '2007', '2008',
                '2009', '2010', '2011', '2012', '2013', '2014','2015', '2016', '2017',
                '2018', '2019', '2020', '2021', '2022']
    if layer == 'Annual Total Precipitation' or layer == 'Monthly Total Precipitation':
        return ['1958','1959','1960','1961','1962','1963','1964','1965','1966','1967',
                '1968','1969','1970','1971','1972','1973','1974','1975','1976','1977',
                '1978','1979','1980','1981','1982','1983','1984','1985','1986','1987',
                '1988','1989','1990','1991','1992','1993','1994','1995','1996','1997',
                '1998','1999','2000','2001','2002','2003','2004','2005','2006','2007',
                '2008','2009','2010','2011','2012','2013','2014','2015','2016','2017',
                '2018','2019','2020']
    if layer == 'NDVI':
        return ['1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', 
                '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', 
                '2017', '2018', '2019', '2020', '2021']
    if layer == 'Landcover':
        return ['1986', '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994', 
                '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', 
                '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', 
                '2013', '2014', '2015', '2016', '2017', '2018']
    if layer == 'Soil Moisture (yearly mean)' or layer == 'Soil Moisture (monthly mean)':
        return ['1982', '1983', '1984', '1985', '1986', '1987', '1988', '1989', '1990', 
                '1991', '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999', 
                '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', 
                '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', 
                '2018', '2019', '2020', '2021', '2022']
    if layer == 'PM2.5 (yearly mean)' or layer == 'PM2.5 (monthly mean)':
        return ['2016', '2017', '2018', '2019', '2020', '2021', '2022']
    if layer == 'Sulphur Dioxide (yearly mean)' or layer == 'Sulphur Dioxide (monthly mean)':
        return ['2018', '2019', '2020', '2021', '2022']
    
layers_monthly = ['Monthly Average Temperature','Monthly Total Precipitation','Soil Moisture (monthly mean)', 
                  'PM2.5 (monthly mean)', 'Sulphur Dioxide (monthly mean)']


# function to generate the year selector according to the selected layer
time_selector = widgets.Output()

# initialize the start_year and end_year as global variable so that their value can be obtained during the visualization
start_year = widgets.Dropdown()
end_year = widgets.Dropdown()
start_month = widgets.Dropdown(
    description='month:',
    options= ['01','02','03','04','05','06','07','08','09','10','11','12'],
    value='01',
    style=style,
    layout= layout_small, 
)
end_month = widgets.Dropdown(
    description='month:',
    options= ['01','02','03','04','05','06','07','08','09','10','11','12'],
    value='01',
    style=style,
    layout= layout_small, 
)

# function to show the year selector with the right range for different selected layer
def getYearSelector(layer):
    time_selector.clear_output()
    timespan = getAvailableYears(layer)
    # dropdown box to select the start year
    global start_year
    start_year = widgets.Dropdown(
        description='Start year:',
        options= timespan,
        value=timespan[0],
        style=style,
        layout= layout_small, 
    )
    # dropdown box to select the end year
    global end_year
    end_year = widgets.Dropdown(
        description='End year:',
        options= timespan,
        value=timespan[len(timespan)-1],
        style=style,
        layout= layout_small, 
    )
    if layer in layers_monthly:   
        with time_selector:
            display(widgets.HBox([start_year,start_month, end_year,end_month]))
            return
    with time_selector:
            display(widgets.HBox([start_year, end_year]))
        

# initialize the year selector
getYearSelector(layer.value)

# callback function for the change of the layer selector's value
def on_layer_change(change): 
    time_selector.clear_output()
    layer = change['new']
    getYearSelector(layer)
    
# when the layer is changed, change the range of the year selector accordingly
layer.observe(on_layer_change, names='value')

# button to visualize the selected layer
show_layer = widgets.Button(
    description='Visualize',
    button_style='primary',
    tooltip='Click to visualize the selected layer',
    style=style,
)

# adjust alignment of the horizontal layout box
box_layout = widgets.Layout(display='flex',
                    flex_flow='row',
                    align_items='center',
                    justify_content = 'flex-start',
                    border='none',
                    width='100%',
                    height = '40px')

# horizontal layout box
hbox1 = widgets.HBox(children = [ time_selector, show_layer], layout = box_layout)
display(layer)
display(hbox1)

Dropdown(description='Select Layer:', options=('Annual Average Temperature', 'Monthly Average Temperature', 'A…

In [3]:
Map = geemap.Map()
Map.set_center(14, 52, 4)
Map

Map(center=[52, 14], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Tog…

In [4]:
# function converting fahrenheit to celsius
def toCelsius(image):
    celsius = image.multiply(0.02).subtract(273.15).copyProperties(image, ['system:time_start'])
    return celsius
# define last day of a month according to different months
end_day_list = {
            '01':'31',
            '02':'28',
            '03':'31',
            '04':'30',
            '05':'31',
            '06':'30',
            '07':'31',
            '08':'31',
            '09':'30',
            '10':'31',
            '11':'30',
            '12':'31',
        }

In [5]:
# visualize the selected layer in the selected time span
def showLayer():
    if layer.value in layers_monthly:
        # if it's a monthly layer set the time span according to the selected year and month value
        start_date = start_year.value + '-'+ start_month.value + '-01'
        end_date = end_year.value + '-'+ end_month.value + '-' + end_day_list[end_month.value]
    else:
        # set the time span according to the selected year value
        start_date = start_year.value + '-01-01'
        end_date = end_year.value + '-12-31'
    # visualize the dataset according to the selected layer
    if layer.value == 'Annual Average Temperature':
        collection = ee.ImageCollection('MODIS/006/MOD11A1').select('LST_Day_1km').filterDate(start_date,end_date)
        collection_celsius = collection.map(toCelsius)
        collection_yearly_mean = geemap.create_timeseries(collection_celsius, start_date, end_date, frequency='year', reducer='mean')
        vis_params = {
            'min': -5,
            'max': 30,
            'opacity': 0.7,
            'palette': [
                '040274', '040281', '0502a3', '0502b8', '0502ce', '0502e6',
                '0602ff', '235cb1', '307ef3', '269db1', '30c8e2', '32d3ef',
                '3be285', '3ff38f', '86e26f', '3ae237', 'b5e22e', 'd6e21f',
                'fff705', 'ffd611', 'ffb613', 'ff8b13', 'ff6e08', 'ff500d',
                'ff0000', 'de0101', 'c21301', 'a71001', '911003'
              ]
        }
        
        Map.add_time_slider(collection_yearly_mean, vis_params, '','Time series of annual average air temperature(℃) at 2m', time_interval=2)
#----------------------------------------------------------------------------------------------------------------------------   
#----------------------------------------------------------------------------------------------------------------------------   
    if layer.value == 'Monthly Average Temperature':
        collection = ee.ImageCollection('MODIS/006/MOD11A1').select('LST_Day_1km').filterDate(start_date,end_date)
        collection_celsius = collection.map(toCelsius)
        collection_monthly_mean = geemap.create_timeseries(collection_celsius, start_date, end_date, frequency='month', reducer='mean')
        vis_params = {
            'min': -20,
            'max': 40,
            'opacity': 0.7,
            'palette': [
                '040274', '040281', '0502a3', '0502b8', '0502ce', '0502e6',
                '0602ff', '235cb1', '307ef3', '269db1', '30c8e2', '32d3ef',
                '3be285', '3ff38f', '86e26f', '3ae237', 'b5e22e', 'd6e21f',
                'fff705', 'ffd611', 'ffb613', 'ff8b13', 'ff6e08', 'ff500d',
                'ff0000', 'de0101', 'c21301', 'a71001', '911003'
              ]
        }
        Map.add_time_slider(collection_monthly_mean, vis_params, '','Time series of monthly average air temperature(℃) at 2m', time_interval=2)
#----------------------------------------------------------------------------------------------------------------------------   
#----------------------------------------------------------------------------------------------------------------------------   
    if layer.value == 'Annual Total Precipitation':
        collection  = ee.ImageCollection('IDAHO_EPSCOR/TERRACLIMATE').select('pr').filterDate(start_date,end_date)
        collection_yearly_sum = geemap.create_timeseries(collection, start_date, end_date, frequency='year', reducer='sum')
        vis_params = {
              'min': 400,
              'max': 1500,
              'opacity': 0.7,
              'palette': ['#f7fcf0','#e0f3db','#ccebc5','#a8ddb5','#7bccc4','#4eb3d3','#2b8cbe','#0868ac','#084081']
            }
        Map.add_time_slider(collection_yearly_sum, vis_params, '','Time series of annual total precipitation(mm)', time_interval=2)
#----------------------------------------------------------------------------------------------------------------------------
#----------------------------------------------------------------------------------------------------------------------------   
    if layer.value == 'Monthly Total Precipitation':
        collection  = ee.ImageCollection('IDAHO_EPSCOR/TERRACLIMATE').select('pr').filterDate(start_date,end_date)
        vis_params = {
              'min': 0,
              'max': 200,
              'opacity': 0.7,
              'palette': ['#f7fcf0','#e0f3db','#ccebc5','#a8ddb5','#7bccc4','#4eb3d3','#2b8cbe','#0868ac','#084081']
            }
        Map.add_time_slider(collection, vis_params, '','Time series of monthly total precipitation(mm)', time_interval=2)
#----------------------------------------------------------------------------------------------------------------------------   
#----------------------------------------------------------------------------------------------------------------------------   
    if layer.value == 'NDVI':
        collection = ee.ImageCollection('LANDSAT/LE07/C01/T1_8DAY_NDVI').select('NDVI').filterDate(start_date,end_date)
        collection_yearly_mean = geemap.create_timeseries(collection, start_date, end_date, frequency='year', reducer='mean')
        vis_params = {
          'min': 0.0,
          'max': 1.0,
          'opacity': 0.7,
          'palette': [
            'FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718', '74A901',
            '66A000', '529400', '3E8601', '207401', '056201', '004C00', '023B01',
            '012E01', '011D01', '011301'
          ],
        }
        Map.add_time_slider(collection_yearly_mean, vis_params, '','Time series of normalized difference vegetation index(NDVI)', time_interval=2)
#----------------------------------------------------------------------------------------------------------------------------   
#----------------------------------------------------------------------------------------------------------------------------   
    if layer.value == 'Landcover':
        collection = ee.ImageCollection('COPERNICUS/CORINE/V20/100m').select('landcover').filterDate(start_date,end_date)
        vis_params = {}
        Map.add_time_slider(collection, vis_params, '','Time series of CORINE landcover', time_interval=2)
#----------------------------------------------------------------------------------------------------------------------------
#----------------------------------------------------------------------------------------------------------------------------   
    if layer.value == 'Soil Moisture (yearly mean)':
        collection  = ee.ImageCollection('NASA/FLDAS/NOAH01/C/GL/M/V001').select('SoilMoi00_10cm_tavg').filterDate(start_date,end_date)
        collection_yearly_mean = geemap.create_timeseries(collection, start_date, end_date, frequency='year', reducer='mean')
        vis_params = {
          'min': 0.0,
          'max': 1.0,
          'opacity': 0.7,
          'palette': ['#ffffe5','#f7fcb9','#d9f0a3','#addd8e','#78c679','#41ab5d','#238443','#006837','#004529']
        }
        Map.add_time_slider(collection_yearly_mean, vis_params, '','Time series of soil moisture (0 - 10 cm underground) in m^3 m^-3', time_interval=2)
#----------------------------------------------------------------------------------------------------------------------------
#----------------------------------------------------------------------------------------------------------------------------   
    if layer.value == 'Soil Moisture (monthly mean)':
        collection  = ee.ImageCollection('NASA/FLDAS/NOAH01/C/GL/M/V001').select('SoilMoi00_10cm_tavg').filterDate(start_date,end_date)
        vis_params = {
          'min': 0.0,
          'max': 1.0,
          'opacity': 0.7,
          'palette': ['#ffffe5','#f7fcb9','#d9f0a3','#addd8e','#78c679','#41ab5d','#238443','#006837','#004529']
        }
        Map.add_time_slider(collection, vis_params, '','Time series of soil moisture (0 - 10 cm underground) in m^3 m^-3', time_interval=2)
#----------------------------------------------------------------------------------------------------------------------------
#----------------------------------------------------------------------------------------------------------------------------   
    if layer.value == 'PM2.5 (yearly mean)':
        collection  = ee.ImageCollection('ECMWF/CAMS/NRT').select('particulate_matter_d_less_than_25_um_surface').filterDate(start_date,end_date)
        collection_yearly_mean = geemap.create_timeseries(collection, start_date, end_date, frequency='year', reducer='mean')
        vis_params = {
          'min': 0.0,
          'max': 7.6e-08,
          'opacity': 0.7,
          'palette': ["5E4FA2","3288BD","66C2A5","ABE0A4","E6F598",
                      "FFFFBF","FEE08B","FDAE61","F46D43","D53E4F","9E0142"]
        }
        Map.add_time_slider(collection_yearly_mean, vis_params, layer_name='Time series of Particulate matter d < 2.5 um(kg m^-3)', time_interval=2)
#----------------------------------------------------------------------------------------------------------------------------
#----------------------------------------------------------------------------------------------------------------------------   
    if layer.value == 'PM2.5 (monthly mean)':
        collection  = ee.ImageCollection('ECMWF/CAMS/NRT').select('particulate_matter_d_less_than_25_um_surface').filterDate(start_date,end_date)
        collection_monthly_mean = geemap.create_timeseries(collection, start_date, end_date, frequency='month', reducer='mean')
        vis_params = {
          'min': 0.0,
          'max': 7.6e-08,
          'opacity': 0.7,
          'palette': ["5E4FA2","3288BD","66C2A5","ABE0A4","E6F598",
                      "FFFFBF","FEE08B","FDAE61","F46D43","D53E4F","9E0142"]
        }
        Map.add_time_slider(collection_monthly_mean, vis_params, layer_name='Time series of Particulate matter d < 2.5 um(kg m^-3)', time_interval=2)
#----------------------------------------------------------------------------------------------------------------------------
#----------------------------------------------------------------------------------------------------------------------------   
    if layer.value == 'Sulphur Dioxide (yearly mean)':
        collection  = ee.ImageCollection('COPERNICUS/S5P/NRTI/L3_SO2').select('SO2_column_number_density').filterDate(start_date,end_date)
        collection_yearly_mean = geemap.create_timeseries(collection, start_date, end_date, frequency='year', reducer='mean')
        vis_params = {
          'min': 0.0,
          'max': 0.0005,
          'opacity': 0.7,
          'palette':['black', 'blue', 'purple', 'cyan', 'green', 'yellow', 'red']
        }
        Map.add_time_slider(collection_yearly_mean, vis_params, layer_name='Time series of SO2 vertical column density at ground level(mol/m^2)', time_interval=2)
#----------------------------------------------------------------------------------------------------------------------------
#----------------------------------------------------------------------------------------------------------------------------   
    if layer.value == 'Sulphur Dioxide (monthly mean)':
        collection  = ee.ImageCollection('COPERNICUS/S5P/NRTI/L3_SO2').select('SO2_column_number_density').filterDate(start_date,end_date)
        collection_monthly_mean = geemap.create_timeseries(collection, start_date, end_date, frequency='month', reducer='mean')
        vis_params = {
          'min': 0.0,
          'max': 0.0005,
          'opacity': 0.7,
          'palette':['black', 'blue', 'purple', 'cyan', 'green', 'yellow', 'red']
        }
        Map.add_time_slider(collection_monthly_mean, vis_params, layer_name='Time series of SO2 vertical column density at ground level(mol/m^2)', time_interval=2)
      

In [6]:
# function to be executed when the visualize button is clicked
def vis_clicked(b):
    # remove current layers and slider control
    if hasattr(Map,'slider_ctrl'):
        # evoke the click event of the close button of the time slider
        Map.slider_ctrl.widget.children[4].click()
    showLayer()
    
# watch the click event of the visualize buttion
show_layer.on_click(vis_clicked)

<a id='timelapse'></a><br>
<strong> &nbsp; &nbsp;Create Timelapse:</strong>

In [7]:
ImgCollList = {
    'Temperature (ERA5)':'ECMWF/ERA5/MONTHLY',
    'Temperature':'MODIS/006/MOD11A1',
    'Precipitation (ERA5)':'ECMWF/ERA5/MONTHLY',
    'Precipitation':'IDAHO_EPSCOR/TERRACLIMATE',
    'NDVI':'LANDSAT/LE07/C01/T1_8DAY_NDVI',
    'Landcover':'COPERNICUS/CORINE/V20/100m',
    'Soil Moisture':'NASA/FLDAS/NOAH01/C/GL/M/V001',
    'PM2.5':'ECMWF/CAMS/NRT',
    'Sulphur Dioxide':'COPERNICUS/S5P/NRTI/L3_SO2',
}
BandList = {
    'Temperature (ERA5)':'mean_2m_air_temperature',
    'Temperature':'LST_Day_1km',
    'Precipitation (ERA5)':'total_precipitation',
    'Precipitation':'pr',
    'NDVI':'NDVI',
    'Landcover':'landcover',
    'Soil Moisture':'SoilMoi00_10cm_tavg',
    'PM2.5':'particulate_matter_d_less_than_25_um_surface',
    'Sulphur Dioxide':'SO2_column_number_density',
}
PaletteList = {
    'Temperature (ERA5)':[
            "#000080","#0000D9","#4000FF","#8000FF","#0080FF","#00FFFF",
            "#00FF80","#80FF00","#DAFF00","#FFFF00","#FFF500","#FFDA00",
            "#FFB000","#FFA400","#FF4F00","#FF2500","#FF0A00","#FF00FF",
        ],
    'Temperature':[
            '040274', '040281', '0502a3', '0502b8', '0502ce', '0502e6',
            '0602ff', '235cb1', '307ef3', '269db1', '30c8e2', '32d3ef',
            '3be285', '3ff38f', '86e26f', '3ae237', 'b5e22e', 'd6e21f',
            'fff705', 'ffd611', 'ffb613', 'ff8b13', 'ff6e08', 'ff500d',
            'ff0000', 'de0101', 'c21301', 'a71001', '911003'
        ],
    'Precipitation (ERA5)':['#f7fcf0','#e0f3db','#ccebc5','#a8ddb5','#7bccc4','#4eb3d3','#2b8cbe','#0868ac','#084081'],
    'Precipitation':['#f7fcf0','#e0f3db','#ccebc5','#a8ddb5','#7bccc4','#4eb3d3','#2b8cbe','#0868ac','#084081'],
    'NDVI':[
        'FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718', '74A901',
        '66A000', '529400', '3E8601', '207401', '056201', '004C00', '023B01',
        '012E01', '011D01', '011301'
        ],
    'Landcover':[],
    'Soil Moisture':['#ffffe5','#f7fcb9','#d9f0a3','#addd8e','#78c679','#41ab5d','#238443','#006837','#004529'],
    'PM2.5':["5E4FA2","3288BD","66C2A5","ABE0A4","E6F598","FFFFBF","FEE08B","FDAE61","F46D43","D53E4F","9E0142"],
    'Sulphur Dioxide':['black', 'blue', 'purple', 'cyan', 'green', 'yellow', 'red'],
}
VisParamList = {
    'Temperature (ERA5)':{
        'bands':['mean_2m_air_temperature'],
        'min': 253,
        'max': 313,
        'palette': [
            "#000080","#0000D9","#4000FF","#8000FF","#0080FF","#00FFFF",
            "#00FF80","#80FF00","#DAFF00","#FFFF00","#FFF500","#FFDA00",
            "#FFB000","#FFA400","#FF4F00","#FF2500","#FF0A00","#FF00FF",
        ]
    },
    'Temperature':{
        'bands':['LST_Day_1km'],
        'min': 13000.0,
        'max': 16000.0,
        'palette':[
            '040274', '040281', '0502a3', '0502b8', '0502ce', '0502e6',
            '0602ff', '235cb1', '307ef3', '269db1', '30c8e2', '32d3ef',
            '3be285', '3ff38f', '86e26f', '3ae237', 'b5e22e', 'd6e21f',
            'fff705', 'ffd611', 'ffb613', 'ff8b13', 'ff6e08', 'ff500d',
            'ff0000', 'de0101', 'c21301', 'a71001', '911003'
        ]
    },
    'Precipitation (ERA5)':{
        'bands':['total_precipitation'],
        'min': 0,
        'max': 0.25,
        'palette': ['#f7fcf0','#e0f3db','#ccebc5','#a8ddb5','#7bccc4','#4eb3d3','#2b8cbe','#0868ac','#084081']
    },
    'Precipitation':{
        'bands':['pr'],
        'min': 0,
        'max': 200,
        'palette': ['#f7fcf0','#e0f3db','#ccebc5','#a8ddb5','#7bccc4','#4eb3d3','#2b8cbe','#0868ac','#084081']
    },
    'NDVI':{
        'bands':['NDVI'],
        'min': 0.0,
        'max': 1.0,
        'palette': [
        'FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718', '74A901',
        '66A000', '529400', '3E8601', '207401', '056201', '004C00', '023B01',
        '012E01', '011D01', '011301'
        ],
    },
    'Landcover':{'bands':['landcover']},
    'Soil Moisture':{
        'bands':['SoilMoi00_10cm_tavg'],
        'min': 0.0,
        'max': 1.0,
        'palette': ['#ffffe5','#f7fcb9','#d9f0a3','#addd8e','#78c679','#41ab5d','#238443','#006837','#004529']
    },
    'PM2.5':{
        'bands':['particulate_matter_d_less_than_25_um_surface'],
        'min': 0.0,
        'max': 7.6e-08,
        'palette': ["5E4FA2","3288BD","66C2A5","ABE0A4","E6F598",
                  "FFFFBF","FEE08B","FDAE61","F46D43","D53E4F","9E0142"]
    },
    'Sulphur Dioxide':{
        'bands':['SO2_column_number_density'],
        'min': 0.0,
        'max': 0.0005,
        'palette':['black', 'blue', 'purple', 'cyan', 'green', 'yellow', 'red']
    },
}
ReducerList = {
    'Temperature (ERA5)':'mean',
    'Temperature':'mean',
    'Precipitation (ERA5)':'sum',
    'Precipitation':'sum',
    'NDVI':'mean',
    'Landcover':'median',
    'Soil Moisture':'mean',
    'PM2.5':'mean',
    'Sulphur Dioxide':'mean',  
}
FrequencyList = {
    'Temperature (ERA5)':['year','month'],
    'Temperature':['year','month'],
    'Precipitation (ERA5)':['year','month'],
    'Precipitation':['year','month'],
    'NDVI':['year','month'],
    'Landcover':['year'],
    'Soil Moisture':['year','month'],
    'PM2.5':['year','month'],
    'Sulphur Dioxide':['year','month'],  
}
TimeSpanList = {
    'Temperature (ERA5)':['1979-01-01','2020-06-01'],
    'Temperature':['2000-02-24','2022-03-18'],
    'Precipitation (ERA5)':['1979-01-01','2020-06-01'],
    'Precipitation':['1958-01-01','2020-12-01'],
    'NDVI':['1999-01-01','2021-10-08'],
    'Landcover':['1986-01-01','2018-12-31'],
    'Soil Moisture':['1982-01-01','2022-02-01'],
    'PM2.5':['2016-06-21','2022-03-18'],
    'Sulphur Dioxide':['2018-07-10','2022-03-19'],  
}
# layer selector dropdown box
dataset = widgets.Dropdown(
    description='Select Dataset:',
    options=['Temperature','Precipitation','Temperature (ERA5)','Precipitation (ERA5)','NDVI','Landcover','Soil Moisture', 'PM2.5','Sulphur Dioxide'],
    value='Temperature',
    style=style
)
dataset

Dropdown(description='Select Dataset:', options=('Temperature', 'Precipitation', 'Temperature (ERA5)', 'Precip…

In [8]:
# Map to show the timelapse
Map1 = geemap.Map()
Map1.set_center(14, 52, 4)
Map1

Map(center=[52, 14], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Tog…

In [9]:
widgets_layout = widgets.Layout(width='30%')
title = widgets.Text(
    value='Timelapse',
    placeholder='Enter title for the timelapse',
    description='Title:',
    disabled=False,
    style=style,
    layout = widgets_layout
)
start_date = widgets.DatePicker(
    description='Start date:',
    value = datetime.strptime(TimeSpanList[dataset.value][0], '%Y-%m-%d'),
    disabled=False,
    style=style,
    layout = widgets_layout
)
end_date = widgets.DatePicker(
    description='End date:',
    value = datetime.strptime(TimeSpanList[dataset.value][1], '%Y-%m-%d'),
    disabled=False,
    style=style,
    layout = widgets_layout
)
speed = widgets.IntSlider(
    value=5,
    min=1,
    max=30,
    step=1,
    description='Frames per second:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d',
    style=style,
    layout = widgets_layout
)
add_progress_bar = widgets.Checkbox(
    value=True,
    description='Add progress bar',
    disabled=False,
    indent=False,
    style=style,
    layout = widgets_layout
)
progress_bar_color = widgets.ColorPicker(
    concise=False,
    description='Progress bar color:',
    value='#0000ff',
    disabled=False,
    style=style,
    layout = widgets_layout
)
font_size = widgets.IntSlider(
    value=25,
    min=10,
    max=50,
    step=1,
    description='Font size:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d',
    style=style,
    layout = widgets_layout
)
font_color = widgets.ColorPicker(
    concise=False,
    description='Font color:',
    value='#ffffff',
    disabled=False,
    style=style,
    layout = widgets_layout
)
frequency = widgets.Dropdown(
    options=FrequencyList[dataset.value],
    value='year',
    description='Temporal frequency:',
    disabled=False,
    style=style,
    layout = widgets_layout
)
submit = widgets.Button(
    description='Create Timelapse',
    button_style='primary',
    tooltip='Click to create timelapse',
    style=style,
)

def on_dataset_change(change):
    dataset = change['new']
    start_date.value = datetime.strptime(TimeSpanList[dataset][0], '%Y-%m-%d')
    end_date.value = datetime.strptime(TimeSpanList[dataset][1], '%Y-%m-%d')
    frequency.options = FrequencyList[dataset]

dataset.observe(on_dataset_change, names='value')

# adjust alignment of the horizontal layout box
box_layout1 = widgets.Layout(display='flex',
                    flex_flow='row',
                    align_items='center',
                    justify_content = 'space-between',
                    border='none',
                    width='100%',
                    height = '40px')

hbox2 = widgets.HBox(children=[title,start_date,end_date],layout = box_layout1)
hbox3 = widgets.HBox([speed,add_progress_bar,progress_bar_color],layout = box_layout1)
hbox4 = widgets.HBox([font_size,font_color,frequency],layout = box_layout1)
hbox5 = widgets.HBox([submit],layout = box_layout1)
vbox = widgets.VBox([hbox2,hbox3,hbox4])
display(vbox)
display(hbox5)

In [10]:
output = widgets.Output()
output

Output()

In [11]:
def submit_clicked(b):
    with output:
        output.clear_output()
        if start_date.value >= end_date.value:
            print('The end date must be later than the start date.')
            return
        feature = Map1.draw_last_feature
        if feature is None:
            print('Please draw a feature on the map to specify the region of interest')
            return
        if dataset.value == 'Landcover':
            print('Timelapse for landcover is not supported for now. Please explore landcover change in the timeslider app above.')
            return
        print('Computing...')
        roi = feature.geometry()
        # out_gif = os.path.join(os.path.expanduser('~'), 'Downloads',title.value+'.gif')
        out_gif = geemap.temp_file_path(".gif")
        # deep copy to avoid changing the original value in the VisParamList
        VisParams = copy.deepcopy(VisParamList[dataset.value])
        # change the min,max value of the visParams when the data range change because of the aggregation frequency
        if dataset.value == 'Precipitation (ERA5)' and frequency.value == 'year':
            VisParams['min'] = 0.4
            VisParams['max'] = 2.0
        if dataset.value == 'Precipitation' and frequency.value == 'year':
            VisParams['min'] = 400
            VisParams['max'] = 1500
        if dataset.value == 'Temperature (ERA5)' and frequency.value == 'year':
            VisParams['min'] = 270
            VisParams['max'] = 300
        if dataset.value == 'Temperature' and frequency.value == 'year':
            VisParams['min'] = 13500.0
            VisParams['max'] = 15500.0
        try:
            #create timelapse
            out_gif = geemap.create_timelapse(
                collection = ImgCollList[dataset.value],
                start_date=start_date.value.strftime("%Y-%m-%d"),
                end_date=end_date.value.strftime("%Y-%m-%d"),
                region=roi,
                frequency=frequency.value,
                reducer=ReducerList[dataset.value],
                date_format='YYYY-MM-dd',
                out_gif=out_gif,
                bands=BandList[dataset.value],
                palette=PaletteList[dataset.value],
                vis_params=VisParams,
                dimensions=768,
                frames_per_second=speed.value,
                crs="EPSG:3857",
                title=title.value,
                title_xy=("2%", "90%"),
                add_text=True,
                text_xy=("2%", "2%"),
                text_sequence=None,
                font_size=font_size.value,
                font_color=font_color.value,
                add_progress_bar=add_progress_bar.value,
                progress_bar_color=progress_bar_color.value,
                progress_bar_height=5,
                loop=0,
            )
            geojson = geemap.ee_to_geojson(roi)
            bounds = geemap.minimum_bounding_box(geojson)
            layer_name = 'timelapse'
            # add the created timelapse to the map as image overlay
            Map1.image_overlay(url=out_gif, bounds=bounds, name=layer_name)
            print('Right click to save the image to your local computer.')
            display(Image(data=open(out_gif,'rb').read(), format='gif')) 
        except:
            empty_text.error(
                "An error occurred while computing the timelapse. You probably requested too much data. Try reducing the ROI or timespan."
            )    
            
submit.on_click(submit_clicked)

<a id='data_source'></a>
<strong>Details about the data source:</strong>

| **Variable**         | **Unit** | **Time availibility**    | **Spatial resolution** | **Data source**                                                                                               |
|----------------------|----------|--------------------------|------------------------|---------------------------------------------------------------------------------------------------------------|
| Temperature          | ℃**      | 2000-02-24 - 2022-03-18* | 1000 m                 | [MOD11A1.006 Terra Land Surface Temperature and Emissivity Daily Global 1km](https://developers.google.com/earth-engine/datasets/catalog/MODIS_006_MOD11A1)                                    |
| Precipitation        | mm       | 1958-01-01 - 2020-12-01* | 4638.3 m               | [TerraClimate: Monthly Climate and Climatic Water Balance for Global Terrestrial Surfaces, University of Idaho](https://developers.google.com/earth-engine/datasets/catalog/IDAHO_EPSCOR_TERRACLIMATE) |
| Temperature (ERA5)   | K        | 1979-01-01 - 2020-06-01* | 27830 m                | [ERA5 Monthly Aggregates - Latest Climate Reanalysis](https://developers.google.com/earth-engine/datasets/catalog/ECMWF_ERA5_MONTHLY)                                                           |
| Precipitation (ERA5) | m        | 1979-01-01 - 2020-06-01* | 27830 m                | [ERA5 Monthly Aggregates - Latest Climate Reanalysis](https://developers.google.com/earth-engine/datasets/catalog/ECMWF_ERA5_MONTHLY)                                                           |
| NDVI                 | /        | 1999-01-01 - 2022-01-01* | 30 m                   | [Landsat 7 Collection 1 Tier 1 8-Day NDVI Composite](https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LE07_C01_T1_8DAY_NDVI)                                                            |
| landcover            | /        | 1986-01-01 - 2018-12-31* | 100 m                  | [Copernicus CORINE Land Cover](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_CORINE_V20_100m)                                                                                  |
| soil moisture        | m3 /m3   | 1982-01-01 - 2022-02-01* | 11132 m                | [FLDAS: Famine Early Warning Systems Network (FEWS NET) Land Data Assimilation System](https://developers.google.com/earth-engine/datasets/catalog/NASA_FLDAS_NOAH01_C_GL_M_V001)                          |
| sulphur dioxide      | mol/m2   | 2018-07-10 - 2022-03-19* | 1113.2 m               | [Sentinel-5P NRTI SO2: Near Real-Time Sulphur Dioxide](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S5P_NRTI_L3_SO2)                                                          |
| PM2.5                | kg/ m3   | 2016-06-21 - 2022-03-18* | 44528 m                | [Copernicus Atmosphere Monitoring Service (CAMS) Global Near-Real-Time](https://developers.google.com/earth-engine/datasets/catalog/ECMWF_CAMS_NRT)                                         |

\* : Current time availability can be different from the one listed here since the GEE datasets might be updated.<br>
\*\*: The unit of the original data is K with a scale of 0.02 and is converted to ℃ in this app.